In [1]:
import sys

sys.path.insert(0, "../../")
from pyecharts import online

online()
#  演示必要的准备代码，使用该库时不需重复此单元格命令

In [2]:
import xalpha as xa
import pandas as pd

# 同一基金的A类C类份额研究

一般来讲A和C份额的区别，就是A收申购费赎回费，C不收，不过每日净值中收取销售服务费，两者的持有平衡点在一年左右

In [3]:
gf500a = xa.fundinfo("162711")
gf500a

广发中证500ETF联接A

In [4]:
gf500c = xa.fundinfo("002903")
gf500c

广发中证500ETF联接C

In [5]:
gf500a.info()

fund name: 广发中证500ETF联接A
fund code: 162711
fund purchase fee: 0.12%
fund redemption fee info: ['小于7天', '1.50%', '大于等于7天，小于1年', '0.50%', '大于等于1年，小于2年', '0.30%', '大于等于2年', '0.00%']


In [6]:
gf500c.info()  # 即使C类，由于国家新规，所有场外基金七天内报复性收取1.5%的赎回费是不可避免的

fund name: 广发中证500ETF联接C
fund code: 002903
fund purchase fee: 0.0%
fund redemption fee info: ['小于等于7天', '1.50%', '大于7天', '0.00%']


至于销售服务费的部分，直接反应在了净值的走势里，下面直接比较两者净值区别

In [7]:
gf500c.bcmkset(gf500a)

In [8]:
gf500c.v_netvalue()
# 由此可见，两者净值高度吻合，但C类对应的红线，随着时间变化，渐渐靠下

In [11]:
gf500c.total_annualized_returns() - gf500c.benchmark_annualized_returns()
# C类的年化收益相交基准A类少了0.18%，和0.2%每年的销售服务费基本吻合

-0.0018000000000000099

下面我们将讨论这两种基金真实持有的时间平衡点，考虑到 xirr 年化计算时，
自带扣除赎回费的虚拟卖出功能，可以直接比较两者 xirr 收益率何时交叉

In [14]:
# 首先制作一个买入并持有的交易单
sta = xa.policy.buyandhold(gf500a, start="2017-01-01")
stc = xa.policy.buyandhold(gf500c, start="2017-01-01")

In [15]:
# 分别实际交易
gf500a_t = xa.trade(gf500a, sta.status)
gf500c_t = xa.trade(gf500c, stc.status)

In [18]:
# 持有半年对比，C类更划算，接近跑赢1%！
gf500a_t.xirrrate("2017-06-30") - gf500c_t.xirrrate("2017-06-30")

-0.00948902317486472

In [22]:
gf500a_t.xirrrate("2018-04-30") - gf500c_t.xirrrate("2018-04-30")

-8.546973222126053e-06

In [26]:
# 由此可知对于广发中证500，持有不同份额的平衡点是17个月左右，超过17个月，A份额更具吸引力
gf500a_t.xirrrate("2018-05-03") - gf500c_t.xirrrate("2018-05-03")

1.6879065194576326e-05

下面我们看下进行网格交易时，两者对收益率的影响

In [28]:
gra = xa.policy.grid(
    gf500a, [3, 3, 3, 3, 3, 3], [5, 5, 5, 5, 5, 5], "2017-01-01", "2018-08-03"
)
grc = xa.policy.grid(
    gf500c, [3, 3, 3, 3, 3, 3], [5, 5, 5, 5, 5, 5], "2017-01-01", "2018-08-03"
)
# 每一仓位都是跌3%入，涨5％卖，总金额都是采用函数默认的100,000

In [29]:
## 实际交易
gfa_grid_t = xa.trade(gf500a, gra.status)
gfc_grid_t = xa.trade(gf500c, grc.status)

In [30]:
gfa_grid_t.xirrrate("2018-04-01"), gfc_grid_t.xirrrate("2018-04-01")
# 截止18年愚人节，采用C份额进行的网格年化收益率比A份额高出了10个百分点！网格当之无愧要选择C类基金

(0.507412374501354, 0.6003244855718297)